In [24]:

from torch.autograd import Function
import load_mnist_data

import torch
import torch.nn as nn
from torch.autograd import Variable
from torch.utils.data import TensorDataset, DataLoader
import torch.optim as optim
import numpy as np
import torch.backends.cudnn as cudnn
import random
import os
import sys


In [31]:
class ReverseLayerF(Function):

    @staticmethod
    def forward(ctx, x, alpha):
        ctx.alpha = alpha

        return x.view_as(x)

    @staticmethod
    def backward(ctx, grad_output):
        output = grad_output.neg() * ctx.alpha

        return output, None

class DANN(nn.Module):
    def __init__(self):
        super(DANN, self).__init__()
        self.feature = nn.Sequential(
                    nn.Conv2d(3, 64, kernel_size=5),
                    nn.BatchNorm2d(64),
                    nn.MaxPool2d(2),
                    nn.ReLU(True),
                    nn.Conv2d(64, 50, kernel_size=5),
                    nn.BatchNorm2d(50),
                    nn.Dropout2d(),
                    nn.MaxPool2d(2),
                    nn.ReLU(True)
                )
                
        self.avgpool=nn.AdaptiveAvgPool2d((5,5))
        self.classifier = nn.Sequential(
                    nn.Linear(50 * 4 * 4, 100),
                    nn.BatchNorm1d(100),
                    nn.ReLU(True),
                    nn.Dropout(),
                    nn.Linear(100, 100),
                    nn.BatchNorm1d(100),
                    nn.ReLU(True),
                    nn.Linear(100, 10),
                )


        self.domain_classifier = nn.Sequential(
                    nn.Linear(50 * 4 * 4, 100),
                    nn.BatchNorm1d(100),
                    nn.ReLU(True),
                    nn.Linear(100, 2),
                )
    def forward(self, input_data, alpha):
        input_data = input_data.expand(input_data.data.shape[0], 3, 28, 28)
        feature = self.feature(input_data)
        feature = feature.view(-1, 50 * 4 * 4)
        reverse_feature = ReverseLayerF.apply(feature, alpha)
        class_output = self.classifier(feature)
        domain_output = self.domain_classifier(reverse_feature)

        return class_output, domain_output


In [25]:

class ReverseLayerF(Function):

    @staticmethod
    def forward(ctx, x, alpha):
        ctx.alpha = alpha

        return x.view_as(x)

    @staticmethod
    def backward(ctx, grad_output):
        output = grad_output.neg() * ctx.alpha

        return output, None


class DANN_with_avg(nn.Module):
    def __init__(self):
        super(DANN, self).__init__()
        self.feature = nn.Sequential(
                    nn.Conv2d(3, 32, kernel_size=5),
                    nn.BatchNorm2d(32),
                    nn.MaxPool2d(2),
                    nn.ReLU(True),
                    nn.Conv2d(32, 48, kernel_size=5),
                    nn.BatchNorm2d(48),
                    nn.Dropout2d(),
                    nn.MaxPool2d(2),
                    nn.ReLU(True)
                )
                
        self.avgpool=nn.AdaptiveAvgPool2d((5,5))
        self.classifier = nn.Sequential(
                    nn.Linear(48*5*5, 100),
                    nn.BatchNorm1d(100),
                    nn.ReLU(True),
                    nn.Dropout(),
                    nn.Linear(100, 100),
                    nn.BatchNorm1d(100),
                    nn.ReLU(True),
                    nn.Linear(100, 10),
                )


        self.domain_classifier = nn.Sequential(
                    nn.Linear(48*5*5, 100),
                    nn.BatchNorm1d(100),
                    nn.ReLU(True),
                    nn.Linear(100, 2),
                )
    def forward(self,x,alpha):
        x = x.expand(x.data.shape[0], 3, 28,28)
        x=self.feature(x)
        x=self.avgpool(x)
        x=torch.flatten(x,1)
        task_predict=self.classifier(x)
        x = ReverseLayerF.apply(x,alpha)
        domain_predict=self.domain_classifier(x)
        return task_predict,domain_predict

In [26]:
def train(source, target, net, criterion, optimizer, epoch, use_cuda=True):
    net.train() # Sets the module in training mode.

    train_loss = 0
    correct_source_label = 0
    correct_source_domain = 0
    correct_target_label = 0
    correct_target_domain = 0
    total = 0
    batch_size = 128

    data_target_iter = iter(target)
    len_dataloader = min(len(source), len(target))

    for batch_idx, (inputs, source_label) in enumerate(source):

        p = float(batch_idx + epoch * len_dataloader) / (100 * len_dataloader)
        alpha = 2. / (1. + np.exp(-10 * p)) - 1

        batch_size = inputs.size(0)
        total += batch_size

        # Feed source image to the network
        source_label = source_label.type(torch.LongTensor)
        domain_label = torch.zeros(batch_size).long()

        if use_cuda:
            inputs, source_label, domain_label = inputs.cuda(), source_label.cuda(), domain_label.cuda()
            
        optimizer.zero_grad()
        inputs, source_label = Variable(inputs), Variable(source_label)
        
        class_output, domain_output = net(inputs, alpha)
        
        _, predicted = torch.max(class_output.data, 1)
        correct_source_label += predicted.eq(source_label.data).cpu().sum().item()
        _, predicted = torch.max(domain_output.data, 1)
        correct_source_domain += predicted.eq(domain_label.data).cpu().sum().item()

        loss_s_label = criterion(class_output, source_label)
        loss_s_domain = criterion(domain_output, domain_label)

        # Feed target image to the network
        target_inputs, target_label = data_target_iter.next()
        domain_label = torch.ones(batch_size).long()
        if use_cuda:
            target_inputs, target_label, domain_label = target_inputs.cuda(), target_label.cuda(), domain_label.cuda()
        
        class_output, domain_output = net(target_inputs, alpha)
        loss_t_domain = criterion(domain_output, domain_label)

        _, predicted = torch.max(class_output.data, 1)
        correct_target_label += predicted.eq(target_label.data).cpu().sum().item()
        _, predicted = torch.max(domain_output.data, 1)
        correct_target_domain += predicted.eq(domain_label.data).cpu().sum().item()

        loss = loss_s_label + loss_s_domain + loss_t_domain
        loss.backward()
        optimizer.step()
    
    return correct_source_label, correct_source_domain, correct_target_label, correct_target_domain, total


In [27]:
loader_source, loader_target = load_mnist_data.get_data_loader(1.0)

In [28]:
criterion = nn.CrossEntropyLoss()

net = DANN_with_avg()
if (torch.cuda.is_available()):
    torch.cuda.manual_seed_all(42)
    cudnn.benchmark = True
    net.cuda()
    criterion = criterion.cuda()
    
optimizer = optim.Adam(net.parameters(), lr=0.001) 

for epoch in range(0, 101):
    sl, sd, tl, td, total = train(loader_source, loader_target, net, criterion, optimizer, epoch) 

    if (epoch % 5 == 0):
        print("e: %d, sl: %f, sd: %f, tl: %f, td: %f" % (epoch, sl/total, sd/total, tl/total, td/total))

e: 0, sl: 0.812700, sd: 0.771933, tl: 0.356917, td: 0.753883
e: 5, sl: 0.924817, sd: 0.723617, tl: 0.543133, td: 0.717817
e: 10, sl: 0.923350, sd: 0.699183, tl: 0.613767, td: 0.692683
e: 15, sl: 0.923067, sd: 0.673167, tl: 0.652200, td: 0.663033
e: 20, sl: 0.921650, sd: 0.662083, tl: 0.659450, td: 0.655600
e: 25, sl: 0.923850, sd: 0.658233, tl: 0.659100, td: 0.650483
e: 30, sl: 0.921950, sd: 0.650933, tl: 0.666117, td: 0.642033
e: 35, sl: 0.925167, sd: 0.644967, tl: 0.679200, td: 0.638533
e: 40, sl: 0.924517, sd: 0.642667, tl: 0.681700, td: 0.642133
e: 45, sl: 0.928100, sd: 0.642783, tl: 0.686800, td: 0.641983
e: 50, sl: 0.926233, sd: 0.639600, tl: 0.695450, td: 0.635617
e: 55, sl: 0.927800, sd: 0.635750, tl: 0.705617, td: 0.631883
e: 60, sl: 0.930483, sd: 0.635367, tl: 0.704117, td: 0.631700
e: 65, sl: 0.932100, sd: 0.636417, tl: 0.705617, td: 0.635317
e: 70, sl: 0.933567, sd: 0.634850, tl: 0.711250, td: 0.638433
e: 75, sl: 0.932433, sd: 0.635183, tl: 0.715167, td: 0.632083
e: 80, sl:

In [32]:
criterion = nn.CrossEntropyLoss()

net = DANN()
if (torch.cuda.is_available()):
    torch.cuda.manual_seed_all(42)
    cudnn.benchmark = True
    net.cuda()
    criterion = criterion.cuda()
    
optimizer = optim.Adam(net.parameters(), lr=0.001) 

for epoch in range(0, 101):
    sl, sd, tl, td, total = train(loader_source, loader_target, net, criterion, optimizer, epoch) 

    if (epoch % 5 == 0):
        print("e: %d, sl: %f, sd: %f, tl: %f, td: %f" % (epoch, sl/total, sd/total, tl/total, td/total))

e: 0, sl: 0.819600, sd: 0.769967, tl: 0.377233, td: 0.735333
e: 5, sl: 0.928450, sd: 0.709883, tl: 0.550650, td: 0.712783
e: 10, sl: 0.928700, sd: 0.674283, tl: 0.603983, td: 0.668767
e: 15, sl: 0.928783, sd: 0.658567, tl: 0.618717, td: 0.653467
e: 20, sl: 0.930283, sd: 0.652083, tl: 0.641283, td: 0.647800
e: 25, sl: 0.931167, sd: 0.644950, tl: 0.654483, td: 0.642567
e: 30, sl: 0.932367, sd: 0.635950, tl: 0.677800, td: 0.626933
e: 35, sl: 0.935750, sd: 0.637617, tl: 0.671233, td: 0.630150
e: 40, sl: 0.935500, sd: 0.634150, tl: 0.685633, td: 0.622383
e: 45, sl: 0.937933, sd: 0.625750, tl: 0.691717, td: 0.619100
e: 50, sl: 0.940200, sd: 0.626750, tl: 0.693767, td: 0.620400
e: 55, sl: 0.941033, sd: 0.628317, tl: 0.708717, td: 0.619383
e: 60, sl: 0.944167, sd: 0.628133, tl: 0.712850, td: 0.618533
e: 65, sl: 0.943583, sd: 0.621883, tl: 0.721150, td: 0.616783
e: 70, sl: 0.945067, sd: 0.625117, tl: 0.722383, td: 0.618517
e: 75, sl: 0.945067, sd: 0.618833, tl: 0.733717, td: 0.613983
e: 80, sl: